-- move s3 buckets to staging

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv
from config.s3_utils import list_objects, download

# Load .env and bucket info
load_dotenv()
S3_BUCKET = os.getenv("S3_BUCKET", "bill-review-prod")

# Local destination folder
DEST_DIR = Path("downloaded_v1_jsons")  # ← CHANGE IF NEEDED
DEST_DIR.mkdir(parents=True, exist_ok=True)

# List of non-recursive S3 prefixes
FLAT_PREFIXES = [
    "data/hcfa_json/valid/mapped/staging/",
    "data/hcfa_json/valid/mapped/",
    "data/hcfa_json/valid/mapped/staging/fails/",
    "data/hcfa_json/valid/mapped/staging/success/",
    "data/hcfa_json/readyforprocess/fails/",
    "data/hcfa_json/readyforprocess/",
    "data/hcfa_json/escalations/"
]

def is_direct_file(key: str, prefix: str) -> bool:
    """Check if key is a direct file under prefix (not nested deeper)"""
    return key.startswith(prefix) and '/' not in key[len(prefix):] and key.endswith('.json')

def download_all_flat_jsons():
    for prefix in FLAT_PREFIXES:
        print(f"🔍 Scanning s3://{S3_BUCKET}/{prefix}")
        keys = list_objects(prefix, bucket=S3_BUCKET)
        json_keys = [k for k in keys if is_direct_file(k, prefix)]

        print(f"→ Found {len(json_keys)} JSON files in {prefix}")

        for key in json_keys:
            local_path = DEST_DIR / Path(key).name
            download(key, str(local_path), bucket=S3_BUCKET)

            print(f"✔ Downloaded {key} → {local_path}")

    print("✅ All eligible JSONs downloaded.")


if __name__ == "__main__":
    download_all_flat_jsons()


🔍 Scanning s3://bill-review-prod/data/hcfa_json/valid/mapped/staging/
→ Found 19 JSON files in data/hcfa_json/valid/mapped/staging/
✔ Downloaded data/hcfa_json/valid/mapped/staging/20241217_1263.json → downloaded_v1_jsons\20241217_1263.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250109_1121.json → downloaded_v1_jsons\20250109_1121.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250112_1031.json → downloaded_v1_jsons\20250112_1031.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250114_1010.json → downloaded_v1_jsons\20250114_1010.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250114_1011.json → downloaded_v1_jsons\20250114_1011.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250129_1088.json → downloaded_v1_jsons\20250129_1088.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250204_1044.json → downloaded_v1_jsons\20250204_1044.json
✔ Downloaded data/hcfa_json/valid/mapped/staging/20250319_105819036.json → downloaded_v1_jsons\20250319_1

-- convert v1 jsons into v2 system

In [4]:
import os
import json
import sqlite3
import tempfile
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from config.s3_utils import upload

# === CONFIG ===
INPUT_DIR = Path("downloaded_v1_jsons")  # ← Folder where you downloaded v1 JSONs
DB_PATH = Path("monolith.db")            # ← Update if your DB is elsewhere

# Load .env and S3 info
load_dotenv()
S3_BUCKET = os.getenv("S3_BUCKET", "bill-review-prod")
OUTPUT_PREFIX = "data/ProviderBills/json/"

def format_charge(val):
    try:
        f = float(str(val).replace('$', '').replace(',', '').strip())
        return f"${f:.2f}"
    except Exception:
        return "$0.00"

def normalize_date(date_str):
    try:
        dt = datetime.strptime(date_str.strip(), "%Y-%m-%d")
        return dt.strftime("%m/%d/%y")
    except Exception:
        return date_str.strip()

def clean_and_insert(json_path, conn):
    with open(json_path, 'r', encoding='utf-8') as f:
        raw = json.load(f)

    bill_id = json_path.stem
    processed_at = datetime.now().isoformat()

    patient_info = raw.get("patient_info", {})
    billing_info = raw.get("billing_info", {})
    service_lines = raw.get("service_lines", [])

    billing_info["total_charge"] = format_charge(billing_info.get("total_charge", "$0.00"))

    for line in service_lines:
        line["charge_amount"] = format_charge(line.get("charge_amount", "$0.00"))
        line["date_of_service"] = normalize_date(line.get("date_of_service", ""))

    # Check for mapped status
    mapping_info = raw.get("mapping_info", {})
    claim_id = mapping_info.get("order_id")
    status = "MAPPED" if claim_id else "REVIEWED"
    action = "to_review" if claim_id else "to_validate"
    total_charge = float(billing_info["total_charge"].replace("$", ""))

    cursor = conn.cursor()

    # Insert ProviderBill
    cursor.execute("""
        INSERT INTO ProviderBill (
            id, status, action, last_error, patient_name, patient_dob, patient_zip,
            billing_provider_name, billing_provider_address, billing_provider_tin,
            billing_provider_npi, total_charge, patient_account_no, claim_id
        ) VALUES (?, ?, ?, NULL, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        bill_id,
        status,
        action,
        patient_info.get("patient_name"),
        patient_info.get("patient_dob"),
        patient_info.get("patient_zip"),
        billing_info.get("billing_provider_name"),
        billing_info.get("billing_provider_address"),
        billing_info.get("billing_provider_tin"),
        billing_info.get("billing_provider_npi"),
        total_charge,
        billing_info.get("patient_account_no"),
        claim_id
    ))

    # Insert service lines
    for line in service_lines:
        charge_amt = float(line["charge_amount"].replace("$", ""))
        modifiers = ",".join(line.get("modifiers", [])) if line.get("modifiers") else ""
        cursor.execute("""
            INSERT INTO BillLineItem (
                provider_bill_id, cpt_code, modifier, units,
                charge_amount, allowed_amount, decision,
                reason_code, date_of_service, place_of_service,
                diagnosis_pointer
            ) VALUES (?, ?, ?, ?, ?, NULL, 'pending', '', ?, ?, ?)
        """, (
            bill_id,
            line.get("cpt_code"),
            modifiers,
            line.get("units"),
            charge_amt,
            line.get("date_of_service"),
            line.get("place_of_service"),
            line.get("diagnosis_pointer")
        ))

    conn.commit()

    # Save cleaned version and upload to S3
    cleaned_json = {
        "provider_bill_id": bill_id,
        "processed_at": processed_at,
        "patient_info": patient_info,
        "billing_info": billing_info,
        "service_lines": service_lines
    }

    with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix=".json", encoding='utf-8') as tmp:
        json.dump(cleaned_json, tmp, indent=2)
        tmp_path = tmp.name

    s3_key = f"{OUTPUT_PREFIX}{bill_id}.json"
    upload(tmp_path, s3_key, bucket=S3_BUCKET)
    os.remove(tmp_path)

    print(f"✔ Imported and uploaded: {bill_id} → s3://{S3_BUCKET}/{s3_key}")

def main():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row

    files = list(INPUT_DIR.glob("*.json"))
    if not files:
        print("⚠️ No JSON files found to import.")
        return

    for file in files:
        try:
            clean_and_insert(file, conn)
        except Exception as e:
            print(f"❌ Failed {file.name}: {e}")

    conn.close()
    print("✅ All files processed.")

if __name__ == "__main__":
    main()


✔ Imported and uploaded: 20241217_1028 → s3://bill-review-prod/data/ProviderBills/json/20241217_1028.json
✔ Imported and uploaded: 20241217_1042 → s3://bill-review-prod/data/ProviderBills/json/20241217_1042.json
✔ Imported and uploaded: 20241217_1060 → s3://bill-review-prod/data/ProviderBills/json/20241217_1060.json
✔ Imported and uploaded: 20241217_1077 → s3://bill-review-prod/data/ProviderBills/json/20241217_1077.json
✔ Imported and uploaded: 20241217_1083 → s3://bill-review-prod/data/ProviderBills/json/20241217_1083.json
✔ Imported and uploaded: 20241217_1087 → s3://bill-review-prod/data/ProviderBills/json/20241217_1087.json
✔ Imported and uploaded: 20241217_1095 → s3://bill-review-prod/data/ProviderBills/json/20241217_1095.json
✔ Imported and uploaded: 20241217_1099 → s3://bill-review-prod/data/ProviderBills/json/20241217_1099.json
✔ Imported and uploaded: 20241217_1106 → s3://bill-review-prod/data/ProviderBills/json/20241217_1106.json
✔ Imported and uploaded: 20241217_1110 → s3://